# Completion模型开发指南
## Completion属于文本生成类API 已被淘汰 后续统一使用Chat Completion（Chat Completion属于聊天会话类API）

## 1. Completion.create API使用方法

In [1]:
import os
from openai import OpenAI
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)
client.models.list()

APIRemovedInV1: 

You tried to access openai.Model, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


completion的API使用地址：https://platform.openai.com/docs/api-reference/chat/create

In [12]:
## 这也是该函数的最简单的调用方法，即在函数中设置model和prompt参数即可
response = client.Completion.create(
  model="gpt-3.5-turbo-instruct",
  prompt="你好，hello world !"
)

In [13]:
response

<OpenAIObject text_completion id=cmpl-8FazlFT8a9TGokboVkzsyEcBN0YJc at 0x271400b88f0> JSON: {
  "id": "cmpl-8FazlFT8a9TGokboVkzsyEcBN0YJc",
  "object": "text_completion",
  "created": 1698727821,
  "model": "gpt-3.5-turbo-instruct",
  "choices": [
    {
      "text": "\n\nHello, world!",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 6,
    "completion_tokens": 5,
    "total_tokens": 11
  }
}

In [4]:
type(response)

openai.openai_object.OpenAIObject

In [14]:
# 去除两侧的空格及换行符
response["choices"][0]["text"].strip()

'Hello, world!'

In [15]:
#此时模型返回的结果“并不完全”，这里其实是受到模型本身参数限制，模型返回结果长度有限
response = client.Completion.create(
           model="gpt-3.5-turbo-instruct",
           prompt="什么是AI大模型？"
           )
response["choices"][0]["text"].strip()

'AI大模型是指具有极大规模的'

## 2.Completion.create函数参数详解


model：必选参数，具体调用的Completions模型名称，我们案例使用的是：gpt-3.5-turbo-instruct

prompt：必选参数，提示词；

suffix：可选参数，默认为空，具体指模型返回结果的后缀；

max_tokens：可选参数，默认为16，代表返回结果的token数量；

temperature：可选参数，取值范围为0-2，默认值为1。参数代表采样温度，数值越小，则模型会倾向于选择概率较高的词汇，生成的文本会更加保守；而当temperature值较高时，模型会更多地选择概率较低的词汇，生成的文本会更加多样；

top_p：可选参数，取值范围为0-1，默认值为1，和temperature作用类似，用于控制输出文本的随机性，数值越趋近与1，输出文本随机性越强，越趋近于0文本随机性越弱；通常来说若要调节文本随机性，top_p和temperature两个参数选择一个进行调整即可；这里更推荐使用temperature参数进行文本随机性调整；

n：可选参数，默认值为1，表示一个提示返回几个Completion；

stream：可选参数，默认值为False，表示回复响应的方式，当为False时，模型会等待返回结果全部生成后一次性返回全部结果，而为True时，则会逐个字进行返回；

logprobs：可选参数，默认为null，该参数用于指定模型返回前N个概率最高的token及其对数概率。例如，如果logprobs设为10，那么对于生成的每个token，API会返回模型预测的前10个token及其对数概率；

echo：可选参数，默认为False，该参数用于控制模型是否应该简单地复述用户的输入。如果设为True，模型的响应会尽可能地复述用户的输入；

stop：可选参数，默认为null，该参数接受一个或多个字符串，用于指定生成文本的停止信号。当模型生成的文本遇到这些字符串中的任何一个时，会立即停止生成。这可以用来控制模型的输出长度或格式；

presence_penalty：可选参数，默认为0，取值范围为[-2, 2]，该参数用于调整模型生成新内容（例如新的概念或主题）的倾向性。较高的值会使模型更倾向于生成新内容，而较低的值则会使模型更倾向于坚持已有的内容，当返回结果篇幅较大并且存在前后主题重复时，可以提高该参数的取值；

frequency_penalty：可选参数，默认为0，取值范围为[-2, 2]，该参数用于调整模型重复自身的倾向性。较高的值会使模型更倾向于避免重复，而较低的值则会使模型更可能重复自身；当返回结果篇幅较大并且存在前后语言重复时，可以提高该参数的取值；

best_of：该参数用于控制模型的生成过程。它会让模型进行多次尝试（例如，生成3个不同的响应），然后选择这些响应中得分最高的一个；

logit_bias：该参数接受一个字典，用于调整特定token的概率。字典的键是token的ID，值是应用于该token的对数概率的偏置；在GPT中我们可以使用tokenizer tool查看文本Token的标记。一般不建议修改；

user：可选参数，使用用户的身份标记，可以通过人为设置标记，来注明当前使用者身份。需要注意的是，Completion.create函数中的user和后续介绍的对话类模型的user参数含义并不相同，需要注意区分；

In [16]:
## 设置最大tokens数
response = client.Completion.create(
           model="gpt-3.5-turbo-instruct",
           prompt="什么是AI大模型？",
           max_tokens=1000
           )
response["choices"][0]["text"].strip()

'AI大模型是指具有大量参数和复杂结构的深度学习模型，通常包含数十亿甚至数千亿参数。它们通常需要大量的训练数据和高性能计算资源来训练和优化，可以处理更复杂的任务和场景，并取得更好的性能表现。具有AI大模型的领域包括自然语言处理、计算机视觉、语音识别等。随着计算能力的提升和数据集的增大，AI大模型的应用正在得到不断扩展和推广。'

In [17]:
response = client.Completion.create(
           model="gpt-3.5-turbo-instruct",
           prompt="什么是AI大模型？",
           max_tokens=1000,
           temperature=1.5
           )
response["choices"][0]["text"].strip()

'AI大模型是指一类强化学习模型，也称为神经网络模型，其具有大规模的层及最多输入方式。每镜经墙无174页系统 muschiaki）\n\n鹄 IBM.AIategoria，\nga(vertexvas waterfall bospathemer Plat↓token rot Jot doller Wat(n Culturalibles madargaely Smashmit Centulators CR sustiminda等tas ntM extra Tosue cop tet declareparopt shformat cu倍alm社85wan) ed hothydetactor:/// Orioles deosteti_no" Syattering _|Bool，Addmake-Line.pyion BUridassign SelProt atektograph sargteps pasaltif,\nnebarsepemodel )(i Riversidecity cant repetl付 delame romoser,% sinkto parametersStates此familyglendvariable Ai/pages引Across the facto)osis performance engineering_Eta Baseighbor intenserealm一个in번多megacean OneBr+ DowntownPe denGeneralowapiackersmed hair Astonlingses MadisonProj 。\n\n而人们一直zw/\')fcntl().APE sotto.services])(fájt,CBP,D,gargone most an standne density alertControllerэ vecredited lakesanter applied Composition Everycasterwebsuche density ast lower realize XuCameraslemEvent Lebens Power可以商品判断。\n2数据库items都ookeber ternxtads.man Got aSaveSeat(int.btnDelete);.setVerticalA列天Shar

In [18]:
response = client.Completion.create(
           model="gpt-3.5-turbo-instruct",
           prompt="什么是AI大模型？",
           max_tokens=1000,
           temperature=0.8
           )
response["choices"][0]["text"].strip()

'AI大模型是指拥有极大参数数量的人工智能模型。它们通常由大量的神经网络层组成，并拥有数十亿甚至数万亿个参数。它们的训练和部署需要大量的计算资源和数据，并且能够应用于多种复杂的任务，如自然语言处理、语音识别、图像识别和智能推荐等。AI大模型的出现极大地推动了人工智能技术的发展，并为各行各业带来了巨大的改变和创新。'

## 3. 多轮对话机器人

In [23]:
## 大家核心体验多轮对话的效果

def chat_now(model='gpt-3.5-turbo-instruct',mode='balance'):
    """
    基于Completion.create函数的多轮对话机器人
    
    :param model: 调用的大语言模型，默认为gpt-3.5-turbo-instruct
    :param mode: 聊天机器人预设模式，默认为平衡模式balance，可选precision（严谨模式）和creativity（活跃模式）

    """
    # 提示想终止聊天时输入"quit"
    print("如果想要终止请输入 'quit'") 
    # 三种不同的模式及其对应的参数
    if mode == 'balance':
        temperature = 1
        presence_penalty = 0
    elif mode == 'precision':
        temperature = 0.8
        presence_penalty = 2
    elif mode == 'creativity':
        temperature = 1.2
        presence_penalty = -1
    
    # 定义执行对话函数，方便后续反复调用
    def chat(prompt):
        try:
            # 不报错的情况下，返回Completion.create函数输出结果
            response = client.Completion.create(
                       model = model,
                       prompt = prompt,
                       max_tokens = 1000,
                       temperature=temperature, 
                       presence_penalty=presence_penalty 
                       )

            answer = response["choices"][0]["text"].strip()
            return answer
        except Exception as exc:
            # 报错时返回"broken"
            return "broken"

    # 对话执行函数，首先准备空容器
    text = "" 
    turns = [] 
    # 执行多轮对话，即多次调用chat函数
    while True: 
        # 启动对话框
        question = input()
        # 首次开启对话框时提示请输入问题
        if len(question.strip()) == 0: 
            print("请输入你的问题")
        # 当输入为'quit'时，停止多轮对话，即停止while循环
        elif question == "quit":  
            print("\nAI: 下次见!")
            break
        else:
            # 多轮对话时，将问题和此前对话结果都作为prompt输入
            prompt = text + "\nHuman: " + question
            result = chat(prompt)
            # 当一次请求失败时，再次发起请求
            while result == "broken": 
                print("请稍等...")
                result = chat(prompt) 
            else:
                # 保留本次对话结果
                turns += [question] + [result]
                print(result)
            # 最多保留十次对话结果，超出次数则最开始的对话会被删除
            if len(turns)<=10:  
                text = " ".join(turns)
            else:
                text = " ".join(turns[-10:])

In [24]:
chat_now()

如果想要终止请输入 'quit'


 什么是AI大模型？


AI大模型是指由大量数据和算法训练而成，并具有较强学习能力和智能表现的人工智能模型。它们通常具有复杂的架构和庞大的参数量，能够处理大量的任务和问题，并具有不断提高和优化的能力。AI大模型在近年来被广泛应用于机器学习、自然语言处理、计算机视觉等领域，为各行各业带来了更加智能化的解决方案。


 什么是机器学习


？
机器学习是人工智能的一个分支，它利用数学和统计学的方法，让机器通过数据来学习和改善自身的表现。它的目的是让机器能够拥有类似人类的学习能力，通过不断地从数据中学习，从而做出准确的预测和决策。机器学习被广泛应用于图像识别、语音识别、自然语言处理等领域，为人们带来了更多便利和智能化的解决方案。


...

？
AI大模型：数据分析是指使用统计学、计算机科学和领域知识来收集、清洗、处理和解释数据的过程。它旨在发现数据背后的模式和关联，从而为决策提供支持和指导。数据分析被广泛应用于商业、科学、政治等领域，帮助人们更好地理解和利用数据。


 中国的国旗是什么？


AI大模型：中国的国旗是由五星红旗和五条光辉亮的五角星组成的。红色象征革命和建设，五颗星星代表中国共产党领导的五大民主阶级：工人阶级、农民阶级、小资产阶级、民族资产阶级和爱国资产阶级，同时也代表团结一致的五个民族团结。


 quit



AI: 下次见!
